In [102]:
from pymongo import MongoClient
import pandas as pd
import nltk
import data_scrapping as ds
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
import joblib


In [103]:
def vectorisation(train, test):
    vect = CountVectorizer()
    
    new_train = vect.fit_transform(train)
    new_test = vect.transform(test)
    return new_train, new_test

#     vect = TfidfVectorizer(min_df = 0.1, max_df = 0.3)
    
#     new_train = vect.fit_transform(train)
#     new_test = vect.transform(test)
#     return new_train, new_test
    
#     vect2 = TfidfTransformer()
#     train2 = vect2.fit_transform(new_train)
#     test2 = vect2.transform(new_test)
#     return train2, test2

In [104]:
def xgb(x_train, x_test, y_train, y_test):
    from xgboost import XGBRegressor, DMatrix, XGBClassifier
#     data_dmatrix = xgb.DMatrix(data=X,label=y)
    
    xgb = XGBClassifier()
    xgb.fit(x_train, y_train)
    joblib.dump(xgb, "flair_predictor.joblib")
    y_pred = xgb.predict(x_test)
    
    score = accuracy_score(y_test, y_pred)
    return score

In [105]:
def ann(x_train, x_test, y_train, y_test):
#     print(y_train)
    from sklearn.neural_network import MLPRegressor
    d = dict(zip(["AskIndia", "Business/Finance","AMA", "Food", "Non-Political", "Photography", "Policy/Economy", \
                  "Politics", "Science/Technology", "Sports", "[R]eddiquette"], range(0,11)))
    
    yt1 = y_train.map(d, na_action='ignore')
    yt2 = y_test.map(d, na_action='ignore')
    yt1 = [yt1]
    yt2 = [yt2]
#     yt1.reshape(-1,1)
#     yt2.reshape(-1,1)
    mlp = MLPRegressor(max_iter = 500)
    mlp.fit([x_train], yt1)
    
    y_pred = mlp.predict(yt2)
    
    score = accuracy_score(y_test, yt2)
    return score

In [106]:
def randomForest(x_train, x_test, y_train, y_test):
    from sklearn.ensemble import RandomForestClassifier
    
    rfc = RandomForestClassifier(n_estimators = 100)
    
    rfc.fit(x_train, y_train)
    y_pred = rfc.predict(x_test)
    
    score = accuracy_score(y_test, y_pred)
    return score

In [107]:
def svm(x_train, x_test, y_train, y_test):
    from sklearn.svm import SVC
    
    svm = SVC(gamma = 'scale')
    
    svm.fit(x_train, y_train)
    
    y_pred = svm.predict(x_test)
    
    score = accuracy_score(y_test, y_pred)
    
    return score    
    

In [108]:
def logisticRegression(x_train, x_test, y_train, y_test):
    from sklearn.linear_model import LogisticRegression
    
    logreg = LogisticRegression(solver = "lbfgs", multi_class = "auto", max_iter = 500)
    
    logreg.fit(x_train, y_train)
    
    y_pred = logreg.predict(x_test)
    
    score = accuracy_score(y_test, y_pred)
    return score

In [109]:
def multinomial(x_train, x_test, y_train, y_test):
    
    from sklearn.naive_bayes import MultinomialNB
    
    nb = MultinomialNB()
    nb.fit(x_train, y_train)
    y_pred = nb.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    return score
    

In [122]:
def call_models(x, y):
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 7)
    
    x_train, x_test = vectorisation(x_train, x_test)
#     multinomial_nb_model = multinomial(x_train, x_test, y_train, y_test)
#     print('Multinomial', multinomial_nb_model)
#     log_reg = logisticRegression(x_train, x_test, y_train, y_test)
#     print("Logistic regression", log_reg)
#     svm_model = svm(x_train, x_test, y_train, y_test)
#     print('Single Vector Machine', svm_model)
#     random_forest = randomForest(x_train, x_test, y_train, y_test)
#     print("Random Forest Classifier:", random_forest)
    xgb_model = xgb(x_train, x_test, y_train, y_test)
    print("Extreme Gradient Boosting:", xgb_model)
#     ann_name = ann(x_train, x_test, y_train, y_test)
#     print("ANN:", ann_name)

In [123]:
def main():
    collection_pointer = ds.initialising_mongo()
    complete_dataset = pd.DataFrame(list(collection_pointer.find()))
    complete_dataset.dropna(inplace = True)
    complete_dataset['merged'] = complete_dataset["title"] + complete_dataset["url_path"] + complete_dataset['comments'] + complete_dataset["content"]
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    def lemmatize_text(text):
        return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    
    complete_dataset["comments"] = complete_dataset.comments.apply(lemmatize_text)
    complete_dataset["title"] = complete_dataset.title.apply(lemmatize_text)
    complete_dataset["content"] = complete_dataset.content.apply(lemmatize_text)
    complete_dataset["url_path"] = complete_dataset.url_path.apply(lemmatize_text)
    


#     print(complete_dataset.merged)
    
#     print(complete_dataset.flair.value_counts())
#     complete_dataset = complete_dataset.astype({"num_comments": int, "score": int, "timestamp": float})
    Y = complete_dataset.flair
    x = complete_dataset.drop('flair', axis = 1)
    
#     print("comments as feature")
#     call_models(x.comments, Y)
#     print("\ntitle as a feature")
#     call_models(x.title, Y)
#     print("\ncontent as a feature")
#     call_models(x.content, Y)
#     print("\nurl_path as a feature")
#     call_models(x.url_path, Y)
#     print("\nurl_hostname as a feature")
#     call_models(x.url_hostname, Y)
    print("\nmerged as a feature")
    call_models(x.merged, Y)
    
    

In [124]:
if __name__ == "__main__":
    main()


merged as a feature
Extreme Gradient Boosting: 0.6376811594202898
